# Movie Recommendation System Using KNN

This notebook implements a movie recommendation system using **K-Nearest Neighbors (KNN)** based on user ratings and movie features. The recommendation system leverages **collaborative filtering** to suggest movies to users based on their preferences and ratings.

## Overview
The recommendation system follows these steps:
1. **Data Loading**: Load the `ratings.csv` and `movies_rc.csv` datasets which contain movie ratings from users and movie details, respectively.
2. **Data Preprocessing**: Clean the data and explore the dataset to get useful insights.
3. **User-Item Matrix Creation**: Create a sparse user-item matrix where each row represents a movie and each column represents a user.
4. **Similarity Calculation**: Use K-Nearest Neighbors (KNN) with cosine similarity to find similar movies based on user ratings.
5. **Movie Recommendations**: Recommend movies to users based on their highest-rated movie, by finding the top `k` similar movies.

## Key Concepts:
- **Collaborative Filtering**: A method of recommendation based on user-item interactions, where the system recommends items based on the preferences of similar users.
- **Cosine Similarity**: A metric used to measure the similarity between two vectors by calculating the cosine of the angle between them.
- **KNN**: An algorithm that finds the k-nearest neighbors to a given point (movie) based on some distance metric (cosine similarity in this case).

### Dataset Information:
- **ratings.csv**: Contains user ratings for movies with columns `userId`, `movieId`, and `rating`.
- **movies_rc.csv**: Contains movie details with columns `movieId` and `title`.





In [3]:
# Importing necessary libraries for data processing, model creation, and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings

# Suppressing future warnings for cleaner output
warnings.simplefilter(action = 'ignore', category = FutureWarning)


ModuleNotFoundError: No module named 'numpy'